# Scraping and Analysis Tasks

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/)
* Collect the latest News Title and Paragraph Text
* Assign the text to variables that you can reference later.

### Data Sources: 

   #### All the data were scraped from the following websites:

* [NASA Mars News Site](https://mars.nasa.gov/news/) -Scraped the latest News Title and Paragraph Text
* [JPL Featured Space Image](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars) - Scraped the image url for the current Featured Mars Image
* [Mars Facts webpage](https://space-facts.com/mars/) -Scraped the table containing facts about the planet including Diameter, Mass, etc
* [USGS Astrogeology site](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) - Scraped high resolution images for each of Mar's hemispheres


In [6]:
# Import Dependencies & Setup
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
from selenium import webdriver
import pandas as pd
import requests
import time
import pymongo

#### Window users (long and old way)

In [185]:
# Set Executable Path & Initialize Chrome Browser

# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

#### MAC users  (long and old way)

In [186]:
# Set Executable Path & Initialize Chrome Browser
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
# browser = Browser("chrome", **executable_path, headless=False)

### For Chrome: Choose the executable path to driver 

In [304]:
# Setup splinter: Looks/Scrape throu many pages instead of just one page 
# In order to run our selenium webdriver automation scripts on chrome and firefox browser, need to have browser drivers
executable_path = {'executable_path': ChromeDriverManager().install()} 
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\cache\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


### For FireFox

In [188]:
# from webdriver_manager.firefox import GeckoDriverManager
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# NASA Mars News

* Scrape the NASA Mars News Site and collect the latest:
    * News Title
    * Paragraph Text 
* Assign the text to variables that you can reference later.

In [189]:
# Open Browser- Visit Nasa news url through splinter module
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

In [240]:
# HTML Object
nasa_html = browser.html

# Parse HTML Results with BeautifulSoup
nasa_soup = BeautifulSoup(nasa_html, 'lxml')

#print(nasa_soup.prettify())

In [193]:
# Scrape HTML & Finding Everything inside ul
nasa_results = nasa_soup.find('ul', class_='item_list')

#nasa_results

### Scrape and collect the latest News "TITLE"

In [244]:
# Retrieve the latest element that contains news title
news_title = nasa_results.find("div", class_ = "content_title").text

# Display scrapped data for Title
print("\n-----------------------------------\n")
print(f"Latest News Title:\n\n{news_title}")
print("\n-----------------------------------\n")


-----------------------------------

Latest News Title:

MOXIE Could Help Future Rockets Launch Off Mars

-----------------------------------



###  Scrape and collect the "Paragraph Text"

In [245]:
# Retrieve the latest element that contains news paragraph
news_parag = nasa_results.find("div", class_ = "article_teaser_body").text

# Display scrapped data paragraph text
print("\n--------------------------\n")
print(f"Paragraph:\n\n{news_parag}")
print("\n---------------------------\n")


--------------------------

Paragraph:

NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.

---------------------------



# JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image 

* Use splinter to navigate the site & find the image url for the current Featured Mars Image 
* assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.


In [146]:
# Open Browser - Visit Mars Space Images through splinter module
jpl_images_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_images_url)

In [147]:
# Click 'Full Image' Button on main page
browser.links.find_by_partial_text('FULL IMAGE').click()

In [148]:
time.sleep(1)

# Click 'more info' button to get to image page
browser.links.find_by_partial_text('more info').click()

In [149]:
# HTML Object 
img_html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(img_html, 'lxml')

In [150]:
#----------------------------------------
# # My first approach to get image link
#----------------------------------------
# results = soup.find_all('figure', class_='lede')
# results
# # Search and Save image url 
# img_url = results[0].a['href']
# img_url

In [151]:
# Use beautiful soup and store results from image in new page to search for image source
img_url = soup.select_one('figure.lede a img').get('src')
img_url

'/spaceimages/images/largesize/PIA18249_hires.jpg'

In [152]:
#set a new variable to use Base URL (JPL main page url)
main_url = 'https://www.jpl.nasa.gov'

#Concatenate main_url & img_url to create an Absolute URL
featured_image_url = (main_url + img_url)
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18249_hires.jpg'

## Featured Image:

<img src ='https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18249_hires.jpg'>

In [153]:
#empty jpg file
f = open('mars_featured_image.jpg', 'wb')

In [154]:
f.close()

# Mars Facts

* Visit the Mars Facts webpage:

* Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [155]:
# Use Pandas to scrape data
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [166]:
# checking table's type
type(tables)

list

In [171]:
# Take first table for Mars Facts
df = tables[0]
df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [172]:
# Rename columns
df.columns = ["Description","Value"]
df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [173]:
# Convert table to html
mars_fact_table = df.to_html()

mars_fact_table.replace('\n', '')

print(mars_fact_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

In [174]:
df.to_html('mars_fact_table.html')

# Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [305]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [309]:
hemisphere_image_urls = []

# Get a list of All the Hemispheres
links = browser.find_by_css("a.product-item h3")

#loop through links and keep count of links
for x in range(len(links)):

    try: 
        # Find element on each loop to avoid a stale element exception
        browser.find_by_css("a.product-item h3")[x].click()

        # Assign the HTML content of the page to a variable & Parse HTML with BeautifulSoup
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        
        # Get Hemisphere Image
        img_url = soup.find("img", class_ = "wide-image")['src']
        
        # Get Hemisphere Title
        title = soup.find("h2", class_ = "title").text

        #Concatenate main_url & img_url to create an Absolute URL
        main_url = 'https://astrogeology.usgs.gov'
        img_url = main_url + img_url

        hemisphere_dict = {"title": title, "img_url": img_url}

        # Append Dictionary objects 'title' & 'img_url' to List
        hemisphere_image_urls.append(hemisphere_dict)
        
        # Click 'Open' button to get to fullsize image
        browser.links.find_by_partial_text('Open').click()
        
        # Navigate Backwards
        browser.back()

    except:
          print("Scraping Complete, Nothing Found")
            
hemisphere_image_urls 

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]